In [1]:
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv

from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
def scrape_cricbuzz(_: str = "") -> str:
    try:
        url = "https://www.cricbuzz.com/cricket-match/live-scores"
        res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(res.text, "html.parser")

        live_boxes = soup.find_all("div", class_="cb-mtch-lst")[:3]
        if not live_boxes:
            return "No live matches found right now."

        summary = ""
        for box in live_boxes:
            header = box.find("a")
            if not header:
                continue
            title = header.text.strip()
            link = "https://www.cricbuzz.com" + header.get("href")
            score_tag = box.find("div", class_="cb-scr-wll-chvrn")
            score = score_tag.text.strip() if score_tag else "Score not available"
            summary += f"🏏 {title}\n{score}\n🔗 {link}\n\n"

        return summary.strip()
    except Exception as e:
        return f"❌ Error scraping Cricbuzz: {e}"

In [ ]:
cricket_tool = Tool(
    name="get_live_cricket_scores",
    func=scrape_cricbuzz,
    description="Scrapes Cricbuzz and returns full live match summaries, including team names, scores, and match status. Use when the user asks for live cricket details."

)

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    google_api_key=GOOGLE_API_KEY
)

agent = initialize_agent(
    tools=[cricket_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

C:\Users\Naman Goyal\AppData\Local\Temp\ipykernel_1976\550119436.py:9: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
query = "Show me the current live cricket scores with details"
response = agent.run(query)

print("\n🏏 Live Cricket Scores:")
print(response)
print(scrape_cricbuzz())

C:\Users\Naman Goyal\AppData\Local\Temp\ipykernel_1976\759244360.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)




> Entering new AgentExecutor chain...
Thought: The user wants live cricket scores.  I should use the `get_live_cricket_scores` function to get the information.
Action: get_live_cricket_scores
Action Input: ""
Observation: 🏏 Qatar vs Saudi Arabia,
QAT101 (17.4 Ovs) SAU100-7 (20 Ovs) Qatar won by 1 run
🔗 https://www.cricbuzz.com/live-cricket-scores/127247/qat-vs-sau-1st-match-saudi-arabia-tour-of-qatar-2025

🏏 Singapore vs Malaysia,
SIN27-4 (5 Ovs) MLY Malaysia opt to bowl
🔗 https://www.cricbuzz.com/live-cricket-scores/127153/sin-vs-mly-3rd-match-asia-pacific-cricket-champions-trophy-2025

🏏 Guyana Amazon Warriors vs Rangpur Riders,
GAW196-3 (20 Ovs) RGR164 (19.5 Ovs) Guyana Amazon Warriors won by 32 runs
🔗 https://www.cricbuzz.com/live-cricket-scores/119531/gaw-vs-rgr-final-global-super-league-2025
Thought:Question: Show me the current live cricket scores with details
Thought:The user wants live cricket scores.  I should use the `get_live_cricket_scores` function to get the informatio